In [1]:
import sys # Python version retriever
import os # file management
import json # Java Script Object Notation parsing and dumping, basically working with dictionaries

import requests # Content download from urls
from urllib.parse import quote as urlencode # deciphering and creating url

from astropy.table import Table # creating loading editing AstroPy Tables (a specifically-modified Pandas DataFrame)
import numpy as np # Numerical Python, basically for everything of matrix operations and calculations

import pprint # Pretty printing to work around limitations of base print() function
pp = pprint.PrettyPrinter(indent=4)

In [2]:
def mast_query(request):
    """Perform a MAST query.

        Parameters
        ----------
        request (dictionary): The MAST request json object

        Returns head,content where head is the response HTTP headers, and content is the returned data"""

    # Base API url
    request_url='https://mast.stsci.edu/api/v0/invoke'

    # Grab Python Version
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    req_string = json.dumps(request)
    req_string = urlencode(req_string)

    # Perform the HTTP request
    resp = requests.post(request_url, data="request="+req_string, headers=headers)

    # Pull out the headers and response content
    head = resp.headers
    content = resp.content.decode('utf-8')

    return head, content

In [3]:
mashup_request = {"service":"Mast.Caom.Filtered",
                  "format":"json",
                  "params":{
                      "columns":"COUNT_BIG(*)",
                      "filters":[
                          {"paramName":"instrument_name",
                           "values":["MIRI/IMAGE"],},
                          {"paramName":"dataRights",
                           "values":["PUBLIC"],},
                          {"paramName":"calib_level",
                           "values":3,}
                      ]}}

headers2, out_string2 = mast_query(mashup_request)
count2 = json.loads(out_string2)

In [ ]:
mashup_request = {"service":"Mast.Caom.Filtered",
                  "format":"json",
                  "params":{
                      "columns":"COUNT_BIG(*)",
                      "filters":[
                          {"paramName":"instrument_name",
                           "values":["MIRI/IMAGE"],},
                          {"paramName":"dataRights",
                           "values":["PUBLIC"],}
                      ]}}

headers, out_string = mast_query(mashup_request)
count = json.loads(out_string)

pp.pprint(count)

In [5]:
# JWST MIRI Special

# Mast.Caom.Filtered is not the only service provided through MAST, JWST MIRI specific Mast.Jwst.Filtered.Miri is one service for MIRI data of JWST

mashup_request = {"service":"Mast.Jwst.Filtered.Miri", # Service is specific for Mid-Infrared INstrument of James Webb Space Telescope
                  "format":"json",
                  "params":{
                      "columns":"*",  # Retrieves all columns
                      "filters":[
                          {"paramName":"instrume", # service specific instrument variable
                           "values":["MIRI"],},
                            {"paramName":"subarray", # type of signal record in subarrays,
                           "values":["SUB256"]}] # here we chose 256x256 subarray
                          }}

headers, out_string = mast_query(mashup_request)
count = json.loads(out_string)

#pp.pprint(count)

In [ ]:
from astropy.table import Table
results = Table(names=('Title', 'Number of Data'), dtype=('str','int32'))

lia = [x.get('title',"") for x in count["data"]] # This retrieves all Proposals involving this type of data
elem = set(lia) # remove duplicates
elem = list(elem) # put back no-duplicate set into a list

for i in elem:
    cou = lia.count(i)
    print(i, "# of appearance: ", cou) # report number of data in each specific proposal with set criteria above
    results.add_row([i,cou])



In [ ]:
# Easy-to-look up reporting of the result above

results.sort('Number of Data')
results.reverse()
results.pprint_all(align='<')

In [ ]:
# Zooming in on a specific Proposal with assigning something to "title" variable, then populating a science_products table and printing all

sci_prod_arr = [x for x in count['data'] if x.get("title", None) == 'JWST MIRI Imaging Survey of Planetary-mass Companions: Testing the Compact Disk Hypothesis']
science_products = Table()

for col, atype in [(x['name'], x['type']) for x in count['fields']]:
    if atype=="string":
        atype="str"
    if atype=="boolean":
        atype="bool"
    if atype=="date":
        atype="str"
    if atype == "int":
        atype = "float" # array may contain nan values, and they do not exist in numpy integer arrays
    science_products[col] = np.array([x.get(col,None) for x in sci_prod_arr], dtype=atype)

print("Number of science products:",len(science_products))
science_products.pprint_all()


In [ ]:
# Bundle downloading the filtered list above

url_list = [("uri", url) for url in science_products['dataURI']]
extension = ".tar.gz"

download_url = 'https://mast.stsci.edu/api/v0.1/Download/bundle'
resp = requests.post(download_url + extension, data=url_list)
out_path = "/content/drive/MyDrive/mastapril/"

out_file = out_path + "mastDownload" + extension

with open(out_file, 'wb') as FLE:
    FLE.write(resp.content)

# Sanity check for file
if not os.path.isfile(out_file):
    print("ERROR: " + out_file + " failed to download.")
else:
    print("COMPLETE: ", out_file)

In [ ]:
# Extracting the downloaded tar.gz file to a specific folder

import tarfile

# open file
file = tarfile.open('/content/drive/MyDrive/mastapril/mastaprilmastDownload.tar.gz')

# extracting file
file.extractall('/content/drive/MyDrive/mastapril/')

file.close()